<a href="https://colab.research.google.com/github/Melsonwang1/CVNL-assignment/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/


'archive (3).zip'     __pycache__  'Untitled (1)'		       Week02_Exercises.ipynb
 hillclimbing.ipynb   RNN.ipynb    'Week01_IntroToPyTorch (1).ipynb'   Week02_PyTorchII.ipynb
 idlmam.py	      Untitled	    Week01_IntroToPyTorch.ipynb        Week07_CNN_II_Exercise.ipynb


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/archive (3).zip" -d "/content/asl_dataset"


Archive:  /content/drive/MyDrive/Colab Notebooks/archive (3).zip
replace /content/asl_dataset/asl_alphabet_test/asl_alphabet_test/A_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/asl_dataset/asl_alphabet_test/asl_alphabet_test/B_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!ls /content/asl_dataset


asl_alphabet_test  asl_alphabet_train


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 5
image_size = 64  # Resize all images to 64x64

In [ ]:
# Data Augmentation and Normalization
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
# Load Dataset
dataset = datasets.ImageFolder('/content/asl_dataset', transform=transform)
class_names = dataset.classes
num_classes = len(class_names)

In [ ]:
# Split Dataset
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

In [ ]:
# Define CNN model (unchanged)
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (image_size // 8) * (image_size // 8), 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [ ]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_losses, val_losses = [], []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))

        # Validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_losses.append(val_loss / len(val_loader))

    return train_losses, val_losses


# Run experiments for SGD, GD, and MBGD
batch_sizes = {'SGD': 1, 'GD': len(train_data), 'MBGD': 64}
results = {}

for name, batch_size in batch_sizes.items():
    # Create DataLoader with specified batch size
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

    # Create new model and optimizer
    model = CNN(num_classes).to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # Train model
    print(f"Training with {name}...")
    train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
    results[name] = (train_losses, val_losses)


# Plot comparison of losses
plt.figure(figsize=(12, 6))
for name, (train_losses, val_losses) in results.items():
    plt.plot(train_losses, label=f"{name} Train Loss")
    plt.plot(val_losses, label=f"{name} Val Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Comparison: SGD, GD, MBGD")
plt.legend()
plt.show()

In [ ]:
def train_model_with_metrics(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_metrics = {'loss': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    val_metrics = {'loss': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        train_loss = 0.0
        y_true_train, y_pred_train = [], []
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            y_true_train.extend(labels.cpu().numpy())
            y_pred_train.extend(preds.cpu().numpy())

        train_metrics['loss'].append(train_loss / len(train_loader))
        train_metrics['accuracy'].append(accuracy_score(y_true_train, y_pred_train))
        train_metrics['precision'].append(precision_score(y_true_train, y_pred_train, average='weighted'))
        train_metrics['recall'].append(recall_score(y_true_train, y_pred_train, average='weighted'))
        train_metrics['f1'].append(f1_score(y_true_train, y_pred_train, average='weighted'))

        # Validation Phase
        model.eval()
        val_loss = 0.0
        y_true_val, y_pred_val = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                y_true_val.extend(labels.cpu().numpy())
                y_pred_val.extend(preds.cpu().numpy())

        val_metrics['loss'].append(val_loss / len(val_loader))
        val_metrics['accuracy'].append(accuracy_score(y_true_val, y_pred_val))
        val_metrics['precision'].append(precision_score(y_true_val, y_pred_val, average='weighted'))
        val_metrics['recall'].append(recall_score(y_true_val, y_pred_val, average='weighted'))
        val_metrics['f1'].append(f1_score(y_true_val, y_pred_val, average='weighted'))

        # Print progress
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {train_metrics['loss'][-1]:.4f}, Val Loss: {val_metrics['loss'][-1]:.4f}")
        print(f"Train Accuracy: {train_metrics['accuracy'][-1]:.4f}, Val Accuracy: {val_metrics['accuracy'][-1]:.4f}\n")

    return train_metrics, val_metrics



In [ ]:
# Train the model
train_metrics, val_metrics = train_model_with_metrics(model, train_loader, val_loader, criterion, optimizer, num_epochs)

# Plot Training vs Validation Metrics
metrics_to_plot = ['loss', 'accuracy', 'precision', 'recall', 'f1']
plt.figure(figsize=(20, 10))
for i, metric in enumerate(metrics_to_plot, 1):
    plt.subplot(2, 3, i)
    plt.plot(train_metrics[metric], label='Train')
    plt.plot(val_metrics[metric], label='Validation')
    plt.title(f"Train vs Validation {metric.capitalize()}")
    plt.xlabel("Epochs")
    plt.ylabel(metric.capitalize())
    plt.legend()

plt.tight_layout()
plt.show()
